首先先导入需要使用到的包

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

配置一些需要使用到的超参数

In [2]:
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

这里采用MNIST数据集，可以通过torchvision获取：

In [6]:
# 获取数据集
train_dataset=torchvision.datasets.MNIST(root='../../data', # 文件夹位置
                                         train=True, # 是否为训练数据
                                         transform=transforms.ToTensor(), # 转化为Tensor
                                         download=True # 下载数据集
                                        )
test_dataset=torchvision.datasets.MNIST(root='../../data', # 文件夹位置
                                         train=False, # 是否为训练数据
                                         transform=transforms.ToTensor(), # 转化为Tensor
                                         download=True # 下载数据集
                                        )

# 构建DataLoader
train_Loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True # 表示打乱顺序
                                        )
test_Loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=True # 表示打乱顺序
                                       )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 9912320/9912422 [04:47<00:00, 15980.37it/s]

Extracting ../../data\MNIST\raw\train-images-idx3-ubyte.gz to ../../data\MNIST\raw



0it [00:00, ?it/s]
 28%|███████████████████████████████▊                                                                                | 8192/28881 [00:00<00:00, 38744.97it/s]
32768it [00:01, 26574.76it/s]                                                                                                                                


Extracting ../../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../../data\MNIST\raw



0it [00:00, ?it/s]
 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 1589248/1648877 [00:07<00:00, 210223.29it/s]

Extracting ../../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data\MNIST\raw




0it [00:00, ?it/s]

  0%|                                                                                                                               | 0/4542 [00:00<?, ?it/s]

8192it [00:01, 6607.31it/s]                                                                                                                                  


Extracting ../../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data\MNIST\raw
Processing...
Done!


构建模型，这里还是用`nn.Linear()`来创建模型即可，但是MNIST数据集是识别0-9十个手写数字，属于分类问题，所以不能再使用MSE作为损失函数，需要使用到交叉熵等分类问题的损失函数。

In [11]:
model=nn.Linear(input_size,num_classes) # 输入是28*28长度的数据，输出是0-9的分类
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

开始训练：

In [13]:
total_step = len(train_Loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_Loader):
        # 由于图像是28*28*1的矩阵，而模型是28*28=784的序列，需要reshape一下
        images = images.reshape(-1, input_size)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2415
Epoch [1/5], Step [200/600], Loss: 2.1421
Epoch [1/5], Step [300/600], Loss: 2.0666
Epoch [1/5], Step [400/600], Loss: 1.9918
Epoch [1/5], Step [500/600], Loss: 1.9023
Epoch [1/5], Step [600/600], Loss: 1.7863
Epoch [2/5], Step [100/600], Loss: 1.7364
Epoch [2/5], Step [200/600], Loss: 1.7174
Epoch [2/5], Step [300/600], Loss: 1.6819
Epoch [2/5], Step [400/600], Loss: 1.6513
Epoch [2/5], Step [500/600], Loss: 1.5890
Epoch [2/5], Step [600/600], Loss: 1.4919
Epoch [3/5], Step [100/600], Loss: 1.4624
Epoch [3/5], Step [200/600], Loss: 1.3701
Epoch [3/5], Step [300/600], Loss: 1.3490
Epoch [3/5], Step [400/600], Loss: 1.2533
Epoch [3/5], Step [500/600], Loss: 1.3034
Epoch [3/5], Step [600/600], Loss: 1.3121
Epoch [4/5], Step [100/600], Loss: 1.2254
Epoch [4/5], Step [200/600], Loss: 1.2649
Epoch [4/5], Step [300/600], Loss: 1.1827
Epoch [4/5], Step [400/600], Loss: 1.0744
Epoch [4/5], Step [500/600], Loss: 1.1200
Epoch [4/5], Step [600/600], Loss:

In [ ]:
测试模型，由于测试的时候不需要计算梯度，这样可以